In [1]:
!pip install dash jupyter-dash


In [1]:
# Install required libraries
!pip install jupyter-dash
!pip install requests
!pip install nest_asyncio

# Import necessary libraries
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import requests
from google.colab import files
import nest_asyncio

# Enable nest_asyncio to allow nested event loops in Colab
nest_asyncio.apply()

# Load data
file1_path = '/aaw1313_data_s1.xlsx'  # Adjust to your file location
file1_main_data = pd.read_excel(file1_path, sheet_name='population change by species NA')

# Initialize Dash App
app = JupyterDash(__name__)

# App layout
app.layout = html.Div([
    html.H1("Endangered Species Visualization", style={'textAlign': 'center'}),

    # Dropdown for category selection
    html.Div([
        html.Label("Select Category:"),
        dcc.Dropdown(
            id='category-dropdown',
            options=[
                {'label': 'Breeding Biome', 'value': 'Breeding.Biome'},
                {'label': 'Bird Group', 'value': 'bird.group'}
            ],
            value='Breeding.Biome'
        ),
    ], style={'width': '40%', 'margin': 'auto'}),

    # Graph output
    dcc.Graph(id='endangered-graph'),
])

# Callback for updating the graph
@app.callback(
    Output('endangered-graph', 'figure'),
    [Input('category-dropdown', 'value')]
)
def update_graph(category):
    threshold_value = 50  # Fixed threshold for percentage loss
    filtered_data = file1_main_data.copy()
    filtered_data["Endangered"] = filtered_data["Loss_med"] <= -(threshold_value / 100) * filtered_data["last_year_popest"]

    # Group and calculate percentages
    summary = filtered_data.groupby(category)["Endangered"].mean() * 100
    summary = summary.reset_index()

    # Create bar plot
    fig = px.bar(
        summary,
        x=category,
        y="Endangered",
        title=f"Percentage of Endangered Species by {category}",
        labels={"Endangered": "Percentage Endangered", category: "Category"},
        color="Endangered",
    )
    return fig

# Run the Dash app in external mode (will be accessible externally in Colab)
app.run_server(mode='external')

# Save the app as an HTML file using requests
def save_html_file():
    # Get the external URL for the Dash app
    url = "http://127.0.0.1:8050"  # This is the default URL for Dash apps in Colab

    # Make a request to get the HTML content of the page
    response = requests.get(url)

    # Save the HTML content to a file
    with open("endangered_species_dashboard.html", "w") as file:
        file.write(response.text)

    # Download the HTML file
    files.download("endangered_species_dashboard.html")

# Call the function to save and download the HTML file
save_html_file()


/usr/local/lib/python3.10/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

Dash app running on:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>